In [197]:
from gymnasium import Env
from gymnasium import Space
from gymnasium.spaces import Discrete
import pandas as pd
import numpy as np
import random as rm

In [198]:
carbon_intensity = pd.read_csv('Datasets/Carbon_intensity_processed_hourly.csv')
electricity_usage = pd.read_csv('Datasets/Household_processed_hourly.csv')

In [199]:
carbon_intensity['Datetime (UTC)'] = pd.to_datetime(carbon_intensity['Datetime (UTC)'], utc=True)
carbon_intensity = carbon_intensity.set_index('Datetime (UTC)')
carbon_intensity.index = carbon_intensity.index.tz_convert(None)
carbon_intensity = carbon_intensity['Carbon Intensity gCO₂eq/kWh (LCA)']


In [200]:
c_int = np.asarray(carbon_intensity['2021-07-05 00:00:00':'2021-12-05 21:00:00'],dtype=np.int32)

In [201]:
electricity_usage['date'] = pd.to_datetime(electricity_usage['date'], utc=True)
electricity_usage= electricity_usage.set_index('date')
electricity_usage.index = electricity_usage.index.tz_convert(None)
electricity_usage = electricity_usage['energy meter val']

In [202]:
e_usage = np.asarray(electricity_usage['2015-07-05 00:00:00':], dtype=np.int32)

In [203]:
energy_matrix = np.vstack((c_int, e_usage)).T
energy_matrix

array([[404,  96],
       [370, 158],
       [388,  81],
       ...,
       [473, 128],
       [475, 128],
       [475, 128]], dtype=int32)

In [204]:
# Constant Environment Variables 
env_length = 24

In [205]:
# def reward_algo(self):
#     if 60 <= self.state <= 65:
#         return 1
#     else:
#         return -1

In [206]:
def is_done(self):
    if self.length == 0:
        return True
    else:
        return False

In [207]:
class CustomObservationSpace(Space):
    def __init__(self, e_matrix):
        super().__init__(dtype=object)
        self.e_matrix = e_matrix
        self.index = 0

    def get_first(self):
        return self.e_matrix[0]
    
    def get_next(self):
        self.index += 1
        return self.e_matrix[self.index]

    def sample(self):
        return np.random.choice(self.e_matrix)
  
    def contains(self, observation):
        # Check if the observation (two valued arr) is valid based on allowed values
        return observation in self.e_matrix
    
observation_space = CustomObservationSpace(energy_matrix)


In [208]:
class Environment(Env):
    def __init__(self):
        # Actions are discharge(-1), nothing (0) and charge (1)
        self.action_space = Discrete(3, start = -1)
        # Custom observation space, contains the arr.length * 2 sized matrix and, later, battery
        self.observation_space = observation_space
        self.state = observation_space.get_first()
        self.length = env_length
        
    def step(self, action):
        print(f'We are at index: {observation_space.index}')
        print(f'The current values are:{observation_space.e_matrix[observation_space.index]}')
        
        # Perform 'game logic' for each possible agent action
        if action == -1:
            print('Discharge')
            print()
        elif action == 0:
            print('Nothing')
            print()
        elif action == 1:
            print('Charge')
            print()
        else:
            print('Should not happen')
        
        # Reduce remaining env size
        self.length -= 1
        # Go to next element of observation space
        self.state = observation_space.get_next()
        
        # Update reward
        # reward = reward_algo(self)
            
        # Check if finished
        done = is_done(self)
        
        return self.state, done
    def render(self):
        # Implement if needed
        pass
    
    def reset(self):
        self.state = observation_space.get_first()
        self.length = env_length

In [209]:
env = Environment()

In [210]:
episodes = 1
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    # score = 0 

    while not done:
        action = env.action_space.sample() # Random actions here
        n_state, done = env.step(action)
        # score+=reward
    # print('Episode:{} Score:{}'.format(episode, score))

We are at index: 0
The current values are:[404  96]
Nothing

We are at index: 1
The current values are:[370 158]
Nothing

We are at index: 2
The current values are:[388  81]
Nothing

We are at index: 3
The current values are:[399 120]
Charge

We are at index: 4
The current values are:[402 136]
Nothing

We are at index: 5
The current values are:[340  90]
Charge

We are at index: 6
The current values are:[321 159]
Nothing

We are at index: 7
The current values are:[311 156]
Nothing

We are at index: 8
The current values are:[300 123]
Nothing

We are at index: 9
The current values are:[299 209]
Discharge

We are at index: 10
The current values are:[297 148]
Nothing

We are at index: 11
The current values are:[294 152]
Charge

We are at index: 12
The current values are:[296 131]
Charge

We are at index: 13
The current values are:[300 149]
Discharge

We are at index: 14
The current values are:[301 158]
Charge

We are at index: 15
The current values are:[318 270]
Charge

We are at index: 16


In [211]:
# states = env.observation_space.shape
# actions = env.action_space.n